In [1]:
# Import all packages
import os
import glob
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
# Set necessary variables to append csv files
path_name = '/Users/thaysmartinez/Documents/Flights Data Exploration/datasets/'
os.chdir(path_name)
extension = 'csv'
csv_files = [i for i in glob.glob('*.{}'.format(extension))]

list_flights = []

# Append csv files into one DataFrame
for file_name in csv_files:
    df = pd.read_csv(file_name, index_col=None, header=0)
    list_flights.append(df)

flights = pd.concat(list_flights, axis=0, ignore_index=True)

In [3]:
pd.set_option('display.max_columns', 25)
flights.head()

,FL_DATE,ORIGIN,ORIGIN_CITY_NAME,DEST,DEST_CITY_NAME,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,...,ARR_DELAY,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 25
0,2019-03-01,CVG,"Cincinnati, OH",MSP,"Minneapolis, MN",1534,1617.0,43.0,28.0,1645.0,1729.0,18.0,...,59.0,0.0,134.0,150.0,104.0,596.0,0.0,0.0,59.0,0.0,0.0,NaN
1,2019-03-01,JFK,"New York, NY",BUF,"Buffalo, NY",940,1035.0,55.0,56.0,1131.0,1228.0,8.0,...,82.0,0.0,94.0,121.0,57.0,301.0,0.0,55.0,27.0,0.0,0.0,NaN
2,2019-03-01,MSN,"Madison, WI",LGA,"New York, NY",1845,1954.0,69.0,14.0,2008.0,2259.0,13.0,...,74.0,0.0,133.0,138.0,111.0,812.0,0.0,0.0,6.0,0.0,68.0,NaN
3,2019-03-01,TYS,"Knoxville, TN",DTW,"Detroit, MI",735,729.0,-6.0,15.0,744.0,855.0,8.0,...,-22.0,0.0,110.0,94.0,71.0,443.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-03-01,JFK,"New York, NY",BUF,"Buffalo, NY",1336,1530.0,114.0,30.0,1600.0,1652.0,6.0,...,105.0,0.0,97.0,88.0,52.0,301.0,35.0,0.0,0.0,0.0,70.0,NaN


In [4]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31746841 entries, 0 to 31746840
Data columns (total 26 columns):
 #   Column               Dtype  
---  ------               -----  
 0   FL_DATE              object 
 1   ORIGIN               object 
 2   ORIGIN_CITY_NAME     object 
 3   DEST                 object 
 4   DEST_CITY_NAME       object 
 5   CRS_DEP_TIME         int64  
 6   DEP_TIME             float64
 7   DEP_DELAY            float64
 8   TAXI_OUT             float64
 9   WHEELS_OFF           float64
 10  WHEELS_ON            float64
 11  TAXI_IN              float64
 12  CRS_ARR_TIME         int64  
 13  ARR_TIME             float64
 14  ARR_DELAY            float64
 15  DIVERTED             float64
 16  CRS_ELAPSED_TIME     float64
 17  ACTUAL_ELAPSED_TIME  float64
 18  AIR_TIME             float64
 19  DISTANCE             float64
 20  CARRIER_DELAY        float64
 21  WEATHER_DELAY        float64
 22  NAS_DELAY            float64
 23  SECURITY_DELAY       float64
 

### Clean

- Columns in capitals
- Unnecessary columns (columns: unnamed: 25)
- Columns with incorrect data type
  - fl_date             : object - datetime
  - crs_dep_time        : int    - datetime (HH:MM)
  - dep_time            : float  - datetime (HH:MM)
  - taxi_out            : float  - datetime (HH:MM)
  - wheels_off          : float  - datetime (HH:MM)
  - wheels_on           : float  - datetime (HH:MM)
  - taxi_in             : float  - datetime (HH:MM)
  - crs_arr_time        : int    - datetime (HH:MM)
  - arr_time            : float  - datetime (HH:MM)
  - crs_elapsed_time    : float  - int
  - actual_elapsed_time : float  - int
  - air_time            : float  - int
  - distance            : float  - int
  - delay_cause         : object - categorical
  - delay duration      : float  - int
- Unnecessary rows (i.e. flights with delays lower than 120 min.)
- Columns as variables (columns: carrier_delay, weather_delay, nas_delay, security_delay, late_aircraft_delay)
- Column delay_cause with trailing **\_delay** string
- City and State in same column (columns: origin_city_name and dest_city_name)
- Columns not descriptive (columns: origin, dest, dep_time, arr_time)
- Columns too descriptive (columns: origin_city_name, dest_city_name)

In [5]:
# Drop rows where relevant information is missing
flights.dropna(axis='index', 
               subset=['CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 
                       'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY'], 
               how='any', 
               inplace=True)

In [6]:
# Filter flights with arrival delay equal or above 120 minutes
flights = flights[flights['ARR_DELAY'] >= 120]

In [7]:
# Melt columns
id_vars = (['FL_DATE', 'ORIGIN', 'ORIGIN_CITY_NAME', 'DEST', 'DEST_CITY_NAME', 
            'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 
            'WHEELS_ON', 'TAXI_IN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY', 
            'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE'])

value_vars = (['CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 
               'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY'])

flights = (pd.melt(flights, id_vars=id_vars, value_vars=value_vars, 
                   var_name='DELAY_CAUSE', value_name='DELAY_DURATION'))

In [8]:
# Replace 0.0 by NaN
flights['DELAY_DURATION'].replace(0.0, np.nan, inplace=True)

# Drop rows which delay_duration is NaN
flights.dropna(axis='index', subset=['DELAY_DURATION'], inplace=True)

In [9]:
# Lowercase columns
flights.columns = map(str.lower, flights.columns)

In [10]:
# Convert flight date to datetime data type
flights['fl_date'] = pd.to_datetime(flights['fl_date'])

In [11]:
# Convert columns from numeric to object types and pad zeroes
columns = ['crs_dep_time', 'dep_time', 'wheels_off', 
           'wheels_on', 'crs_arr_time', 'arr_time']

converter = (lambda x: x.astype(str)
                        .str.replace(r'(\.0)$', '')
                        .str.zfill(4)
                        .str.replace('2400', '0000')   
             if x.name in columns else x)

flights = flights.apply(converter)

In [12]:
# Convert columns to time timestamp
for column in columns:
    flights[column] = flights[column].apply(lambda x: datetime.strptime(x, '%H%M').time())
    # flights[column] = pd.to_datetime(flights[column], format='%H%M').dt.time

In [13]:
# Transform string in 'delay_cause' column
capitalize_nas = lambda x : x.upper() if x == 'Nas' else x

flights['delay_cause'] = (flights['delay_cause'].str.replace('_DELAY', '')
                                                .str.replace('_', ' ')
                                                .str.title()
                                                .apply(capitalize_nas))

In [14]:
# Convert to int
columns = ['taxi_out', 'taxi_in', 'dep_delay', 
           'arr_delay', 'crs_elapsed_time', 'actual_elapsed_time', 
           'air_time', 'distance', 'delay_duration']

converter = lambda x: x.astype(int) if x.name in columns else x

flights = flights.apply(converter)

In [15]:
# Rename columns
flights.rename(columns={'origin': 'origin_airport', 
                        'dest': 'dest_airport', 
                        'origin_city_name': 'origin_city', 
                        'dest_city_name': 'dest_city', 
                        'dep_time': 'actual_dep_time', 
                        'arr_time': 'actual_arr_time'},
               errors='raise', inplace=True)

In [16]:
# Split columns
flights[['origin_city','origin_state']] = flights.origin_city.str.split(',', expand=True)
flights[['dest_city','dest_state']] = flights.dest_city.str.split(',', expand=True)

In [17]:
# Reorder columns
ordered_columns = ['fl_date', 'origin_airport', 'origin_city', 'origin_state',
                   'dest_airport', 'dest_city', 'dest_state', 'crs_dep_time', 
                   'actual_dep_time', 'dep_delay', 'taxi_out', 
                   'wheels_off', 'wheels_on', 'taxi_in', 
                   'crs_arr_time', 'actual_arr_time', 'arr_delay', 
                   'crs_elapsed_time', 'actual_elapsed_time', 
                   'air_time', 'distance', 'delay_cause', 'delay_duration']

flights = flights[ordered_columns]

In [18]:
flights.head()

,fl_date,origin_airport,origin_city,origin_state,dest_airport,dest_city,dest_state,crs_dep_time,actual_dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,actual_arr_time,arr_delay,crs_elapsed_time,actual_elapsed_time,air_time,distance,delay_cause,delay_duration
2,2019-03-01,ATL,Atlanta,GA,TYS,Knoxville,TN,12:19:00,15:28:00,189,16,15:44:00,16:17:00,2,13:15:00,16:19:00,184,56,51,33,152,Carrier,184
6,2019-03-01,LGA,New York,NY,ORF,Norfolk,VA,16:59:00,19:13:00,134,36,19:49:00,20:44:00,15,18:34:00,20:59:00,145,95,106,55,296,Carrier,22
7,2019-03-01,ATL,Atlanta,GA,XNA,Fayetteville,AR,13:51:00,16:02:00,131,14,16:16:00,16:51:00,4,14:50:00,16:55:00,125,119,113,95,589,Carrier,119
9,2019-03-01,LGA,New York,NY,PIT,Pittsburgh,PA,11:00:00,12:41:00,101,67,13:48:00,14:55:00,21,12:58:00,15:16:00,138,118,155,67,335,Carrier,1
10,2019-03-01,IND,Indianapolis,IN,RDU,Raleigh/Durham,NC,17:35:00,19:34:00,119,10,19:44:00,21:03:00,11,19:10:00,21:14:00,124,95,100,79,489,Carrier,21


In [19]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1272355 entries, 2 to 3625729
Data columns (total 23 columns):
 #   Column               Non-Null Count    Dtype         
---  ------               --------------    -----         
 0   fl_date              1272355 non-null  datetime64[ns]
 1   origin_airport       1272355 non-null  object        
 2   origin_city          1272355 non-null  object        
 3   origin_state         1272355 non-null  object        
 4   dest_airport         1272355 non-null  object        
 5   dest_city            1272355 non-null  object        
 6   dest_state           1272355 non-null  object        
 7   crs_dep_time         1272355 non-null  object        
 8   actual_dep_time      1272355 non-null  object        
 9   dep_delay            1272355 non-null  int64         
 10  taxi_out             1272355 non-null  int64         
 11  wheels_off           1272355 non-null  object        
 12  wheels_on            1272355 non-null  object        
 1

In [20]:
# Write to csv
dest_folder = '/Users/thaysmartinez/Documents/Flights Data Exploration/'
flights.to_csv('%sflights.%s' % (dest_folder, extension), index=False, encoding='utf-8')